# Feature Engineering Pipeline using Tensorflow Extended (TFX)


We will build up to the feature engineering part. Specifically, We will:
  - ingest data from a base directory with `ExampleGen`
  - compute the statistics of the training data with `StatisticsGen`
  - infer a schema with `SchemaGen`
  - detect anomalies in the evaluation data with `ExampleValidator`
  - preprocess the data into features suitable for model training with `Transform`

## Setup

### Import packages

In [ ]:
!pip install tfx

In [ ]:
import pandas as pd
import tensorflow as tf
import tfx
from tfx import v1 as tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

Using TensorFlow backend
TensorFlow version: 2.13.1
TFX version: 1.14.0


In [ ]:
df = pd.read_csv("./SeoulBikeData.csv", encoding = 'latin1')
df.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


### Preview the dataset
- We will be using the [Seoul Bike Sharing](https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand) dataset.
- Variable Information
 - Date : year-month-day
 - Rented Bike count - Count of bikes rented at each hour
 - Hour - Hour of he day
 - Temperature-Temperature in Celsius
 - Humidity - %
 - Windspeed - m/s
 - Visibility - 10m
 - Dew point temperature - Celsius
 - Solar radiation - MJ/m2
 - Rainfall - mm
 - Snowfall - cm
 - Seasons - Winter, Spring, Summer, Autumn
 - Holiday - Holiday/No holiday
 - Functional Day - NoFunc(Non Functional Hours), Fun(Functional hours)



### Define paths

In [ ]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './seoulbikesharingdemand'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'SeoulBikeData.csv')

In [ ]:
# save the data to a .csv file
df.to_csv(_data_filepath)

### Create the Interactive Context

-  We will be manually executing the pipeline components. For that, TFX provides the Interactive Context so we can step through each component and inspect its outputs.
- We will initialize the `InteractiveContext` below. This will create a database in the `_pipeline_root` directory which the different components will use to save or get the state of the component executions. We can think of it as the data store that makes it possible for the different pipeline components to work together.

In [ ]:
# Initialize the InteractiveContext with a local sqlite file.
# If you leave `_pipeline_root` blank, then the db will be created in a temporary directory.
context = InteractiveContext(pipeline_root=_pipeline_root)

## Run TFX components interactively

### ExampleGen


We will start the pipeline with the [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) component. This  will:

*   split the data into training and evaluation sets (by default: 2/3 train, 1/3 eval).
*   convert each data row into `tf.train.Example` format. This [protocol buffer](https://developers.google.com/protocol-buffers) is designed for Tensorflow operations and is used by the TFX components.
*   compress and save the data collection under the `_pipeline_root` directory for other components to access. These examples are stored in `TFRecord` format. This optimizes read and write operations within Tensorflow especially if you have a large collection of data.

Its constructor takes the path to your data source/directory. In our case, this is the `_data_root` path. The component supports several data sources such as CSV, tf.Record, and BigQuery. Since our data is a CSV file, we will use [CsvExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/CsvExampleGen) to ingest the data.



In [ ]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

We can execute the component by calling the `run()` method of the `InteractiveContext`.

In [ ]:
# Execute the component
context.run(example_gen)

We will notice that an output cell showing the execution results is automatically shown. This metadata is recorded into the database created earlier. This allows you to keep track of your project runs. For example, if you run it again, you will notice the `.execution_id` incrementing.

The output of the components are called *artifacts* and you can see an example by navigating through  `.component.outputs > ['examples'] > Channel > ._artifacts > [0]` above. It shows information such as where the converted data is stored (`.uri`) and the splits generated (`.split_names`).


- We can also examine the output artifacts programmatically with the code below.

In [ ]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: /content/drive/MyDrive/MLOps Notebooks/pipeline/CsvExampleGen/examples/1


- As mentioned, the ingested data is stored in the directory shown in the `uri` field. It is also compressed using `gzip` and we can verify by running the cell below.

In [ ]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

ls: cannot access '/content/drive/MyDrive/MLOps': No such file or directory
ls: cannot access 'Notebooks/pipeline/CsvExampleGen/examples/1/Split-train': No such file or directory


- Since the data collection is saved in [TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord), you will need to use methods that work with that data type. You will need to unpack the individual examples from the `TFRecord` file and format it for printing. Let's do that in the following cells:

In [ ]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''

    # initialize an empty list
    records = []

    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):

        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()

        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()

        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)

        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))

        # append to the records list
        records.append(example_dict)

    return records

In [ ]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

print(type(sample_records[0]))

# Print the output
pp.pprint(sample_records)

<class 'dict'>
[{'features': {'feature': {'': {'int64List': {'value': ['0']}},
                           'Date': {'bytesList': {'value': ['MDEvMTIvMjAxNw==']}},
                           'Dew point temperature(°C)': {'floatList': {'value': [-17.6]}},
                           'Functioning Day': {'bytesList': {'value': ['WWVz']}},
                           'Holiday': {'bytesList': {'value': ['Tm8gSG9saWRheQ==']}},
                           'Hour': {'int64List': {'value': ['0']}},
                           'Humidity(%)': {'int64List': {'value': ['37']}},
                           'Rainfall(mm)': {'floatList': {'value': [0.0]}},
                           'Rented Bike Count': {'int64List': {'value': ['254']}},
                           'Seasons': {'bytesList': {'value': ['V2ludGVy']}},
                           'Snowfall (cm)': {'floatList': {'value': [0.0]}},
                           'Solar Radiation (MJ/m2)': {'floatList': {'value': [0.0]}},
                           'Temper

-  `ExampleGen` has finished ingesting the data, the next step is data analysis.

### StatisticsGen

- The [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) component computes statistics over your dataset for data analysis, as well as for use in downstream components (i.e. next steps in the pipeline).

- `StatisticsGen` takes as input the dataset we just ingested using `CsvExampleGen`.

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

- We can display the statistics with the `show()` method.

In [ ]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

-  A schema defines the expected bounds, types, and properties of the features in our dataset.
- `SchemaGen` will take as input the statistics that we generated with `StatisticsGen`, looking at the training split by default.

In [ ]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

- We can then visualize the generated schema as a table.

In [ ]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'',INT,required,,-
'Date',BYTES,required,,-
'Dew point temperature(°C)',FLOAT,required,,-
'Functioning Day',STRING,required,,'Functioning Day'
'Holiday',STRING,required,,'Holiday'
'Hour',INT,required,,-
'Humidity(%)',INT,required,,-
'Rainfall(mm)',FLOAT,required,,-
'Rented Bike Count',INT,required,,-


,Values
Domain,
'Functioning Day',"'No', 'Yes'"
'Holiday',"'Holiday', 'No Holiday'"
'Seasons',"'Autumn', 'Spring', 'Summer', 'Winter'"


- The next step in the pipeline and see if there are any anomalies in the data.

### ExampleValidator

- The [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component detects anomalies in our data based on the generated schema from the previous step.
- `ExampleValidator` will take as input the statistics from `StatisticsGen` and the schema from `SchemaGen`. By default, it compares the statistics from the evaluation split to the schema from the training split.

In [ ]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

- As with the previous component. We can also visualize the anomalies as a table

In [ ]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

- With no anomalies detected, we can proceed to the next step in the pipeline.

### Transform

The [Transform](https://www.tensorflow.org/tfx/guide/transform) component performs feature engineering datasets. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) library.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module containing the preprocessing function.

In this section, We will work on an example of a user-defined Transform code. The pipeline needs to load this as a module so you need to use the magic command `%% writefile` to save the file to disk. Let's first define a few constants that group the data's attributes according to the transforms we will perform later. This file will also be saved locally.

In [ ]:
# Set the constants module filename
_bikesharing_constants_module_file = 'bikesharing_constants.py'

In [ ]:
%%writefile {_bikesharing_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = [
    'Functioning Day', 'Holiday', 'Seasons'
]

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['Humidity(%)', 'Rainfall(mm)', 'Snowfall (cm)', 'Solar Radiation (MJ/m2)', 'Temperature(°C)', 'Visibility (10m)', 'Wind speed (m/s)']

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['Hour']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'Hour': 4}

# Feature that the model will predict
LABEL_KEY = 'Rented Bike Count'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing bikesharing_constants.py


- Next, we will work on the module that contains preprocessing_fn().This function defines how you will transform the raw data into features that your model can train on (i.e. the next step in the pipeline).

In [ ]:
# Set the transform module filename
_bikesharing_transform_module_file = 'bikesharing_transform.py'

In [ ]:
%%writefile {_bikesharing_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import bikesharing_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = bikesharing_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = bikesharing_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = bikesharing_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = bikesharing_constants.FEATURE_BUCKET_COUNT
_LABEL_KEY = bikesharing_constants.LABEL_KEY
_transformed_name = bikesharing_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])

    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.scale_to_0_1(inputs[_LABEL_KEY])

    return outputs

Writing bikesharing_transform.py


- We can now pass the training data, schema, and transform module to the `Transform` component

In [ ]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_bikesharing_transform_module_file))

# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

- Examine the output artifacts of `Transform` (i.e. `.component.outputs` from the output cell above). This component produces several outputs:

* `transform_graph` is the graph that can perform the preprocessing operations.
* `transformed_examples` points to the preprocessed training and evaluation data.
* `updated_analyzer_cache` are stored calculations from previous runs.

- Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.

In [ ]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['metadata', 'transform_fn', 'transformed_metadata']

* The `metadata` subdirectory contains the schema of the original data.
* The `transformed_metadata` subdirectory contains the schema of the preprocessed data.
* The `transform_fn` subdirectory contains the actual preprocessing graph.

We can also take a look at the first three transformed examples using the helper function defined earlier.

In [ ]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Functioning Day_xf': {'int64List': {'value': ['0']}},
                           'Holiday_xf': {'int64List': {'value': ['0']}},
                           'Hour_xf': {'int64List': {'value': ['0']}},
                           'Humidity(%)_xf': {'floatList': {'value': [0.37755102]}},
                           'Rainfall(mm)_xf': {'floatList': {'value': [0.0]}},
                           'Rented Bike Count_xf': {'floatList': {'value': [0.07431246]}},
                           'Seasons_xf': {'int64List': {'value': ['0']}},
                           'Snowfall (cm)_xf': {'floatList': {'value': [0.0]}},
                           'Solar Radiation (MJ/m2)_xf': {'floatList': {'value': [0.0]}},
                           'Temperature(°C)_xf': {'floatList': {'value': [0.21616872]}},
                           'Visibility (10m)_xf': {'floatList': {'value': [1.0]}},
                           'Wind speed (m/s)_xf': {'floatList': {'value': [0.30136988]}}}}},
 {'featur

## Conclusion

- Tensorflow Extended (TFX) provides components that work together to execute the most common steps in a machine learning project. We have now executed data ingestion, validation, and transformation components in our pipeline.We can get hands-on practice as well with training and model evaluation.